start

Linear Programming

In [2]:
import requests
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pulp import *
import numpy as np

In [3]:
data_api = "http://127.0.0.1:8000/api/ingredients/datatoai"
response = requests.get(data_api)
data_json = response.json()
data = pd.DataFrame(data_json)
data.head()

,id,name,serving_size,price,calories,protein,carb,fat,sat_fat,trans_fat,...,cholesterol,sodium,calcium,iron,zinc,status,image,rate,created_at,updated_at
0,1,gạo lứt,100,4000,111,2.6,23.0,0.9,0.200,0.0,...,0,5,10,4.0,0.90,active,images/dataseeds/gaolut.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z
1,2,ức gà,100,7500,165,31.0,0.0,3.6,1.000,0.0,...,73,74,8,7.0,0.60,active,images/dataseeds/ucga.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z
2,3,tôm,100,11000,99,24.0,0.0,0.3,0.100,0.0,...,189,414,38,12.0,1.40,active,images/dataseeds/tom.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z
3,4,bông cải xanh,100,3500,34,2.8,7.0,0.4,0.100,0.0,...,0,33,47,73.0,0.41,active,images/dataseeds/bongcaixanh.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z
4,5,cà rốt,100,2000,41,0.9,19.0,0.2,0.003,0.0,...,0,69,33,0.3,0.24,active,images/dataseeds/carrot.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z


In [4]:
def getData(data_api):
    response = requests.get(data_api)
    data_json = response.json()
    data = pd.DataFrame(data_json)
    data = data.replace('na',np.nan)
    data = data.fillna(0)
    return data

In [5]:
data = getData("http://127.0.0.1:8000/api/ingredients/datatoai")

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            40 non-null     int64  
 1   name          40 non-null     object 
 2   serving_size  40 non-null     int64  
 3   price         40 non-null     int64  
 4   calories      40 non-null     int64  
 5   protein       40 non-null     float64
 6   carb          40 non-null     float64
 7   fat           40 non-null     float64
 8   sat_fat       40 non-null     float64
 9   trans_fat     40 non-null     float64
 10  fiber         40 non-null     float64
 11  sugar         40 non-null     float64
 12  cholesterol   40 non-null     int64  
 13  sodium        40 non-null     int64  
 14  calcium       40 non-null     int64  
 15  iron          40 non-null     float64
 16  zinc          40 non-null     float64
 17  status        40 non-null     object 
 18  image         40 non-null     ob

In [7]:
# data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_nutrition /data.csv')
data = data.replace('na',np.nan)
data = data.fillna(0)

Model

In [8]:
def build_nutritional_values(calories):
    protein_calories = round(calories*30)/100
    carb_calories = round(calories*35)/100
    fat_calories = round(calories*35)/100
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [9]:
def extract_gram(table):
    protein_grams = round(table['Protein Calories']/4.)
    carbs_grams = round(table['Carbohydrates Calories']/4.)
    fat_grams = round(table['Fat Calories']/9.)
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [28]:
extract_gram(build_nutritional_values(500))

{'Protein Grams': 38, 'Carbohydrates Grams': 44, 'Fat Grams': 19}

In [65]:
days_data = data
def model(days_data,calories, max_ingredients):
    G = extract_gram(build_nutritional_values(calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data
    # day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.iloc[:,4].tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0.0, upBound=1.5, cat='Continuous')
    food_chosen = pulp.LpVariable.dicts("Chosen",indices = food,lowBound = 0,upBound = 1,cat='Integer')
    e = day_data.iloc[:,6].tolist()
    f = day_data.iloc[:,7].tolist()
    p = day_data.iloc[:,5].tolist()
    sat_f = day_data.iloc[:,8].tolist()
    cholesterol = day_data.iloc[:,12].tolist()
    trans_f = day_data.iloc[:,9].tolist()
    fiber = day_data.iloc[:,10].tolist()
    sugar = day_data.iloc[:,11].tolist()
    sodium = day_data.iloc[:,13].tolist()
    calium = day_data.iloc[:,14].tolist()
    fe = day_data.iloc[:,15].tolist()
    zn = day_data.iloc[:,16].tolist()
    # v_a = day_data.vitamin_c.tolist()
    # v_b = day_data.vitamin_a_rae.tolist()
    # v_c = day_data.calcium.tolist()
    # v_d = day_data.irom.tolist()
    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(x))]  )<= (calories + 200)
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(x))]  )>= (calories - 200)
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>= E # = 250 gram carb
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>= F  # = 80 gram fat
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>= P # = 70 gram protein
    prob += pulp.lpSum( [x[food[i]]*sat_f[i] for i in range(len(x)) ] )>=(13.0*35)/100
    prob += pulp.lpSum( [x[food[i]]*cholesterol[i] for i in range(len(x)) ] )>=10.0
    prob += pulp.lpSum( [x[food[i]]*cholesterol[i] for i in range(len(x)) ] )<=100.0
    # prob += pulp.lpSum( [x[food[i]]*trans_f[i] for i in range(len(x)) ] )>= 1.0
    prob += pulp.lpSum( [x[food[i]]*trans_f[i] for i in range(len(x)) ] )<= 2.2
    # prob += pulp.lpSum( [x[food[i]]*fiber[i] for i in range(len(x)) ] )>=(900.0*35)/100
    prob += pulp.lpSum( [x[food[i]]*sugar[i] for i in range(len(x)) ] )<= 10.0
    prob += pulp.lpSum( [x[food[i]]*sodium[i] for i in range(len(x)) ] )<=round((2300.0*35)/100)
    prob += pulp.lpSum( [x[food[i]]*calium[i] for i in range(len(x)) ] )>=round((1300.0*35)/100)
    prob += pulp.lpSum( [x[food[i]]*fe[i] for i in range(len(x)) ] )>=round((8.7*35)/100)
    prob += pulp.lpSum( [x[food[i]]*zn[i] for i in range(len(x)) ] )>=round((8*35)/100)

    for f in food:
      prob += x[f]>= food_chosen[f]*0.1
      prob += x[f]<= food_chosen[f]*1e5
    # prob += food_chosen['gạo lứt'] >=1
    # prob += x['gạo lứt'] >=1.0
    result = pd.DataFrame([
            {
                'id': 1,
                'name': 'gạo trắng',
                'calories': 200,
                # Other attributes...
            },
            {
                'id': 2,
                'name': 'gạo lứt',
                'calories': 300,
                # Other attributes...
            },
            # Add more dishes...
        ])
    # result['name']
    prob += lpSum([food_chosen[p] for p in result['name']]) >= len(result['name'])
    prob.solve()

    optimal_values = [round(x[food[i]].varValue,2) for i in range(len(food))]

    day_data['OptimalValue'] = optimal_values
    
    if (prob.status != 1):
       return []

    return day_data[day_data["OptimalValue"] > 0]

In [66]:
sol_monday = model(data,500, 3)

In [68]:
result = pd.DataFrame([
            {
                'id': 1,
                'name': 'gạo trắng',
                'calories': 200,
                # Other attributes...
            },
            {
                'id': 2,
                'name': 'gạo lứt',
                'calories': 300,
                # Other attributes...
            },
            # Add more dishes...
        ])
len(result)

2

In [67]:
sol_monday

,id,name,serving_size,price,calories,protein,carb,fat,sat_fat,trans_fat,...,sodium,calcium,iron,zinc,status,image,rate,created_at,updated_at,OptimalValue
0,1,gạo lứt,100,4000,111,2.6,23.0,0.9,0.2,0.0,...,5,10,4.0,0.9,active,images/dataseeds/gaolut.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z,0.10
29,30,thịt bò thăn,100,12000,250,26.2,0.0,17.0,6.8,0.7,...,56,7,2.6,5.1,active,images/dataseeds/thitbothan.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z,1.16
30,31,đậu hũ,100,2000,144,8.1,4.8,8.0,1.2,0.0,...,8,350,2.4,1.6,active,images/dataseeds/dauhu.jpg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z,1.26
37,38,gạo trắng,100,3000,148,7.0,80.0,0.7,0.2,0.0,...,1,10,0.8,0.2,active,images/dataseeds/comtrang.jpeg,0,2023-06-02T14:52:40.000000Z,2023-06-02T14:52:40.000000Z,0.45


In [40]:
sol_monday[sol_monday["OptimalValue"]!= 0]

TypeError: list indices must be integers or slices, not str

In [42]:
food = days_data.name.tolist()
c  = days_data.iloc[:,4].tolist()
x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0.0, upBound=1.5, cat='Continuous')

In [14]:
G = extract_gram(build_nutritional_values(2000))
E = G['Carbohydrates Grams']
F = G['Fat Grams']
P = G['Protein Grams']
day_data = days_data
food = day_data.name.tolist()
c  = day_data.iloc[:,4].tolist()
x  = LpVariable.dicts( "x", indices = food, lowBound=0.0, upBound=1.5, cat='Continuous')
food_chosen = LpVariable.dicts("Chosen",indices = food,lowBound = 0,upBound = 1,cat='Integer')
e = day_data.iloc[:,6].tolist()
f = day_data.iloc[:,7].tolist()
p = day_data.iloc[:,5].tolist()
sat_f = day_data.iloc[:,8].tolist()
cholesterol = day_data.iloc[:,12].tolist()
trans_f = day_data.iloc[:,9].tolist()
fiber = day_data.iloc[:,10].tolist()
sugar = day_data.iloc[:,11].tolist()
sodium = day_data.iloc[:,13].tolist()
calium = day_data.iloc[:,14].tolist()
fe = day_data.iloc[:,15].tolist()
zn = day_data.iloc[:,16].tolist()
prob  = LpProblem( "Diet", LpMinimize )
prob += lpSum( [x[food[i]]*c[i] for i in range(len(x))]  )
prob += lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>= E # = 250 gram carb
prob += lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>= F  # = 80 gram fat
prob += lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>= P # = 70 gram protein
prob += lpSum( [x[food[i]]*sat_f[i] for i in range(len(x)) ] )>=(13.0*35)/100
prob += lpSum( [x[food[i]]*cholesterol[i] for i in range(len(x)) ] )>=10.0
prob += lpSum( [x[food[i]]*cholesterol[i] for i in range(len(x)) ] )<=100.0
# prob += pulp.lpSum( [x[food[i]]*trans_f[i] for i in range(len(x)) ] )>= 1.0
prob += lpSum( [x[food[i]]*trans_f[i] for i in range(len(x)) ] )<= 2.2
# prob += pulp.lpSum( [x[food[i]]*fiber[i] for i in range(len(x)) ] )>=(900.0*35)/100
prob += lpSum( [x[food[i]]*sugar[i] for i in range(len(x)) ] )<= 10.0
prob += lpSum( [x[food[i]]*sodium[i] for i in range(len(x)) ] )<=round((2300.0*35)/100)
prob += lpSum( [x[food[i]]*calium[i] for i in range(len(x)) ] )>=round((1300.0*35)/100)
prob += lpSum( [x[food[i]]*fe[i] for i in range(len(x)) ] )>=round((8.7*35)/100)
prob += lpSum( [x[food[i]]*zn[i] for i in range(len(x)) ] )>=round((8*35)/100)

for f in food:
    prob += x[f]>= food_chosen[f]*0.1
    prob += x[f]<= food_chosen[f]*1e5
prob += food_chosen['gạo lứt'] >=1
prob += x['gạo lứt'] >=1.0
prob.solve()

optimal_values = [round(x[food[i]].varValue,2) for i in range(len(food))]

day_data['OptimalValue'] = optimal_values

PulpSolverError: PuLP: Error while executing glpsol.exe

In [ ]:

food = day_data.name.tolist()
c  = day_data.iloc[:,4].tolist()
e = day_data.iloc[:,6].tolist()
f = day_data.iloc[:,7].tolist()
p = day_data.iloc[:,5].tolist()
sat_f = day_data.iloc[:,8].tolist()
cholesterol = day_data.iloc[:,12].tolist()
trans_f = day_data.iloc[:,9].tolist()
fiber = day_data.iloc[:,10].tolist()
sugar = day_data.iloc[:,11].tolist()
sodium = day_data.iloc[:,13].tolist()
calium = day_data.iloc[:,14].tolist()
fe = day_data.iloc[:,15].tolist()
zn = day_data.iloc[:,16].tolist()
c

In [23]:
prob.solve()

PulpSolverError: PuLP: Error while executing glpsol.exe

In [26]:
def calculate_intake_values(calories, gender = "male"):
    intake_values = {}

    if gender.lower() == 'male':
        intake_values['saturated_fat'] = 13 * (calories / 2000)
        intake_values['trans_fat'] = 2 * (calories / 2000)
        intake_values['cholesterol'] = 300 * (calories / 2000)
        intake_values['sugar'] = 36 * (calories / 2000)
        intake_values['sodium'] = 2300 * (calories / 2000)
        intake_values['calcium'] = 1000 * (calories / 2000)
        intake_values['iron'] = 8 * (calories / 2000)
        intake_values['zinc'] = 11 * (calories / 2000)
    elif gender.lower() == 'female':
        intake_values['saturated_fat'] = 13 * (calories / 2000)
        intake_values['trans_fat'] = 2 * (calories / 2000)
        intake_values['cholesterol'] = 300 * (calories / 2000)
        intake_values['sugar'] = 25 * (calories / 2000)
        intake_values['sodium'] = 2300 * (calories / 2000)
        intake_values['calcium'] = 1000 * (calories / 2000)
        intake_values['iron'] = 18 * (calories / 2000)
        intake_values['zinc'] = 8 * (calories / 2000)

    return intake_values

# Example usage
calories_input = 611
gender_input = 'male'
intake_values = calculate_intake_values(calories_input, gender_input)

In [27]:
intake_values['sugar']

10.998